In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [3]:
opened_fidelity_file = '/Users/sanathnair/Developer/trading/data/fidelity/fd_splits_08_29.html'
unopened_fidelity_file = '/Users/sanathnair/Developer/trading/data/fidelity/fd_splits_08_29_unopened.html'

In [4]:
def _handle_opened_data(opened):
    df = pd.read_html(opened)

    # get the data from the individual split dfs and put them into a list
    prices = []
    for idx, temp in enumerate(df):
        splits = temp.iloc[:-1].to_numpy()
        length = splits.shape[0]
        identifier = np.empty((length, 1))
        identifier.fill(idx)
        updated = np.hstack((splits, identifier))
        prices.append(updated)

    # combine all the rows into one
    res = prices[0]
    for x in prices[1:]:
        res = np.append(res, x, axis = 0)

    # create a df with split info
    splits_df = pd.DataFrame(res, columns = ["Date", "Broker Executed", "Price", "Quantity",
                                             "Dollar Amt", "Identifier"])
    return splits_df

In [5]:
df = _handle_opened_data(opened_fidelity_file)

/opt/homebrew/Caskroom/miniforge/base/envs/trading/lib/python3.9/site-packages/pandas/io/html.py:674: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(udoc, features="html5lib", from_encoding=from_encoding)


ValueError: No tables found

In [ ]:
df

In [ ]:
df[df["Identifier"].duplicated()]

In [ ]:
with open(unopened_fidelity_file) as file:
    soup = BeautifulSoup(file, 'html.parser')
    class_to_find = "gridRow" 
    data = soup.find_all(class_=class_to_find)
data

In [ ]:
temp = data[0]


In [ ]:
new_soup = BeautifulSoup(str(temp), "html.parser")
element = soup.find('div', class_='pvd-grid__grid pvd-grid__grid--default-column-span-12')
text = element.get_text(strip=True).split()


In [ ]:
text

In [ ]:
action = text[0]
symbol = text[4]

In [ ]:
action, symbol

In [ ]:
unopened_df = pd.DataFrame()
with open(unopened_fidelity_file) as file:
    soup = BeautifulSoup(file, 'html.parser')
    class_to_find = "pvd-grid__grid pvd-grid__grid--default-column-span-12" 
    data = soup.find_all(class_=class_to_find)
    for row in data:
        text = row.get_text(strip=True).split()
        row_info = pd.Series([text[0], text[4]], index = ["Action", "Symbol"])
        unopened_df = pd.concat([unopened_df, row_info.to_frame().T], ignore_index = True)
        

In [ ]:
unopened_df

In [ ]:
unopened_df = unopened_df[(unopened_df["Action"] == "Buy") | (unopened_df["Action"] == "Sell")]

In [ ]:
df1 = pd.merge(left = df, right=unopened_df, left_on = "Identifier", right_index = True)
df1

In [ ]:
df1["Broker Executed"] = pd.to_datetime(df1["Broker Executed"], format = '%I:%M:%S %p ET', utc = False) - pd.Timedelta(hours = 3)
df1["Price"] = df1["Price"].str.slice(start = 1)
df1["Dollar Amt"] = df1["Dollar Amt"].str.slice(start = 1)
df1

In [ ]:
file = "/Users/sanathnair/Developer/trading/data/fidelity/fd_splits_08_17.csv"
df = pd.read_csv(file)
df

In [ ]:
df["Broker Executed"] = pd.to_datetime(df["Broker Executed"])

In [ ]:
df.dtypes

In [ ]:
df["Broker Executed"] = df["Broker Executed"].dt.strftime("%X")

In [ ]:
df

In [ ]:
files = [
    "/Users/sanathnair/Developer/trading/data/fidelity/fd_splits_08_17.csv",
    "/Users/sanathnair/Developer/trading/data/fidelity/fd_splits_08_18.csv",
    "/Users/sanathnair/Developer/trading/data/fidelity/fd_splits_08_21.csv",
    "/Users/sanathnair/Developer/trading/data/fidelity/fd_splits_08_22.csv",
    "/Users/sanathnair/Developer/trading/data/fidelity/fd_splits_08_24.csv",
    "/Users/sanathnair/Developer/trading/data/fidelity/fd_splits_08_25.csv",
]
for file in files:
    df = pd.read_csv(file)
    df = df.rename(columns = {
        'Quantity' : "Size"
    })
    df.to_csv(file, index = False)

In [ ]:
df = pd.read_csv("/Users/sanathnair/Downloads/all_fd_trades.csv")

In [ ]:
df = df.drop(columns = ['Unnamed: 0'])

In [ ]:
grouped = df.groupby("Date")

In [ ]:
dfs = {date: group for date, group in grouped}

In [ ]:
for date, df in dfs.items():
    parts = date.split('/')
    df.to_csv(f'report_0{parts[0]}_{parts[1]}.csv', index = False)

In [ ]:
dfs['7/10/2023'].isna().any(axis = 1)

In [ ]:
# 4:14 script stopped came back at 4:52
# left at 6:45 for walk started again 9:30

In [2]:
file = "/Users/sanathnair/Developer/trading/data/fidelity/fd_splits_08_17.csv"

In [4]:
df = pd.read_csv(file)

In [38]:
df

,Date,Broker Executed,Price,Size,Dollar Amt,Identifier,Symbol,Action
0,08/17/2023,10:02:45,80.4200,1.0,80.42,0.0,GWRE,Sell
1,08/17/2023,10:02:12,1.3000,78.0,101.40,1.0,PXLW,Sell
2,08/17/2023,10:01:40,332.8000,1.0,332.80,2.0,GS,Sell
3,08/17/2023,10:01:08,21.2950,5.0,106.48,3.0,SIX,Sell
4,08/17/2023,09:56:50,80.5565,1.0,80.56,4.0,GWRE,Buy
...,...,...,...,...,...,...,...,...
103,08/17/2023,06:50:51,29.3500,3.0,88.05,101.0,BAC,Sell
104,08/17/2023,06:46:47,2.2127,46.0,101.78,102.0,NNBR,Buy
105,08/17/2023,06:46:22,2.4600,42.0,103.32,103.0,PFMT,Buy
106,08/17/2023,06:45:57,7.1933,14.0,100.71,104.0,DISH,Buy


In [45]:
df["Size"].astype(int)

0       1
1      78
2       1
3       5
4       1
       ..
103     3
104    46
105    42
106    14
107     3
Name: Size, Length: 108, dtype: int64